In [ ]:
#-*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from sympy import *
import openai
import itertools
import re
import time

In [ ]:
x = symbols("x")
y = symbols("y")
sym = " -> "

# Training

In [ ]:
# import data
data = pd.read_excel('math_data.xlsx', sheet_name="L30",usecols="A,B", header=None, names=['Q','A'])

# shuffle the rows
df = data.sample(frac=1, random_state=925)

# split dataset
train = df.iloc[0:5]
cv = df.iloc[5:17]
test = df.iloc[17:]

# list used to iterate through all permutations
ls = list(itertools.combinations(list(range(0,5)),3))
temp_cv = pd.DataFrame(columns = ["Examples","T9"])
temp_test = pd.DataFrame(columns = ["Examples","T9"])
results = pd.DataFrame(columns = ["Examples","Question","Right Answer","Generated Answer"])

In [ ]:
openai.api_key = ""


def generate_res(s):
    response = openai.Completion.create(
        model="text-davinci-002",
        prompt=s,
        temperature=0.9,
        max_tokens=50,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0)
    return response["choices"][0]["text"]

In [ ]:
def acc_calculation(ans_df):
    corr_num = 0
    for i in range(len(ans_df)):
        # check if generated answer contains 2 equations and no characters other than x, y, digits and operators
        ans = ans_df.iloc[i, 1].replace(" ","")
        ans = ans.replace("^","**").split(";")
        invalid_char = re.findall(r'[^\^xy0-9=+-/\*;\(\) ]+', ans_df.iloc[i, 1])
        if "," in ans_df.iloc[i, 1]:
            continue
        elif len(ans)!=2 or len(invalid_char)!=0:
            continue 
        elif ans[1] == "":
            continue

        # proceed with valid answers
        # filling in asterisks
        temp = re.findall(r'[xy]|\d|[=+-/\*\(\)]', ans[0])
        j = 1
        while j<len(temp):
            if temp[j] == "x" or temp[j] == "y" or temp[j] == "(":
                if temp[j-1].isdigit() or temp[j-1] == ")" or temp[j-1] == "x" or temp[j-1] == "y":
                    temp.insert(j,"*")
                if temp[j-1] == ".":
                    temp.insert(j,"0*")
            elif temp[j].isdigit() and temp[j-1] == ")":
                temp.insert(j,"*")
            if temp[j] == "." and j+1 == len(temp):
                temp.pop(j)
            j +=1
        first = "".join(temp).split("=")
        j = 1
        temp = re.findall(r'[xy]|\d|[=+-/\*\(\)]', ans[1])
        while j<len(temp):
            if temp[j] == "x" or temp[j] == "y" or temp[j] == "(":
                if temp[j-1].isdigit() or temp[j-1] == ")" or temp[j-1] == "x" or temp[j-1] == "y":
                    temp.insert(j,"*")
                if temp[j-1] == ".":
                    temp.insert(j,"0*")
            elif temp[j].isdigit() and temp[j-1] == ")":
                temp.insert(j,"*")
            if temp[j] == "." and j+1 == len(temp):
                temp.pop(j)
            j +=1
        second = "".join(temp).split("=")
        if "" in first or "" in second:
            continue

        # create equations
        eq1 = Eq(eval(first[0]),eval(first[1]),evaluate=False)
        eq2 = Eq(eval(second[0]),eval(second[1]),evaluate=False)
        reference = ans_df.iloc[i, 0].split(";")
        one = reference[0].split("=")
        two = reference[1].split("=")
        ref1 = Eq(eval(one[0]),eval(one[1]),evaluate=False)
        ref2 = Eq(eval(two[0]),eval(two[1]),evaluate=False)
        
        # check if the model switched variables for representation
        temp = re.findall(r'[xy]|\d|[=+-/\*\(\)]', "=".join([first[0],first[1]]))
        k=0
        while k<len(temp):
            if temp[k] == "x":
                temp[k] = "y"
            elif temp[k] == "y":
                temp[k] = "x"
            k += 1
        sub1 = "".join(temp).split("=")
        temp = re.findall(r'[xy]|\d|[=+-/\*\(\)]', "=".join([second[0],second[1]]))
        k=0
        while k<len(temp):
            if temp[k] == "x":
                temp[k] = "y"
            elif temp[k] == "y":
                temp[k] = "x"
            k += 1
        sub2 = "".join(temp).split("=")
        se1 = Eq(eval(sub1[0]),eval(sub1[1]),evaluate=False)
        se2 = Eq(eval(sub2[0]),eval(sub2[1]),evaluate=False)
        # compare equations
        if solve(eq1)==solve(ref1) and solve(eq2)==solve(ref2):
            corr_num += 1
        elif solve(eq1)==solve(ref2) and solve(eq2)==solve(ref1):
            corr_num += 1
        elif solve(se1)==solve(ref1) and solve(se2)==solve(ref2):
            corr_num += 1
        elif solve(se1)==solve(ref2) and solve(se2)==solve(ref1):
            corr_num += 1
    # return ratio
    return(corr_num/len(ans_df))

In [ ]:
# zero-shot learning
for i in range(len(df)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon.\n"+df.iloc[i,0]
    resp = generate_res(s).replace(" ", "")
    resp = resp.replace("\n", "")
    results.loc[len(results)] = ["None", df.iloc[i,0], df.iloc[i,1], resp]
    time.sleep(1.5)

In [ ]:
# one-shot learning
for i in range(len(train)):
    for j in range(len(cv)):
        s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the example:\n1."\
            +train.iloc[i,0]+sym+train.iloc[i,1]+"\n2."+cv.iloc[j,0]+sym
        resp = generate_res(s).replace(" ", "")
        resp = resp.replace("\n", "")
        results.loc[len(results)] = [train.iloc[i].name, cv.iloc[j,0], cv.iloc[j,1], resp]
        time.sleep(1.5)

In [ ]:
# two-shot learning
combs = list(itertools.combinations(list(range(0,5)),2))
for x,y in combs:
    for i in range(len(cv)):
        s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +train.iloc[x,0]+sym+train.iloc[x,1]+"\n2."\
            +train.iloc[y,0]+sym+train.iloc[y,1]+"\n3."+cv.iloc[i,0]+sym
        resp = generate_res(s)
        resp = resp.replace("\n","")
        results.loc[len(results)] = [(train.iloc[x].name, train.iloc[y].name), cv.iloc[i,0], cv.iloc[i,1], resp]
        time.sleep(1.5)

In [ ]:
# three-shot learning
for x,y,z in ls:
    for i in range(len(cv)):
        s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +train.iloc[x,0]+sym+train.iloc[x,1]+"\n2."\
            +train.iloc[y,0]+sym+train.iloc[y,1]+"\n3."\
            +train.iloc[z,0]+sym+train.iloc[z,1]+"\n4."+cv.iloc[i,0]+sym
        resp = generate_res(s)
        resp = resp.replace("\n","")
        results.loc[len(results)] = [(train.iloc[x].name, train.iloc[y].name, train.iloc[z].name), cv.iloc[i,0], cv.iloc[i,1], resp]
        time.sleep(1.5)

In [ ]:
# zero-shot prediction accuracy on whole dataset
temp_cv.loc[len(temp_cv)] = ["None", acc_calculation(results.iloc[0:30,2:])]

# one-shot prediction accuracy on cv
gp = results.iloc[30:90].groupby("Examples")
for gp_name, group in gp:
    temp_cv.loc[len(temp_cv)] = [str(gp_name), acc_calculation(group.iloc[:,2:])]
    
# two-shot prediction accuracy on cv
gp = results.iloc[90:210].groupby("Examples")
for gp_name, group in gp:
    temp_cv.loc[len(temp_cv)] = [str(gp_name), acc_calculation(group.iloc[:,2:])]

# three-shot prediction accuracy on cv
gp = results.iloc[210:].groupby("Examples")
for gp_name, group in gp:
    temp_cv.loc[len(temp_cv)] = [str(gp_name), acc_calculation(group.iloc[:,2:])]

In [ ]:
temp_cv

# Testing

In [ ]:
# one-shot learning with example 17
one_shot_1 = pd.DataFrame(columns=["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
        s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the example:\n1."\
            +data.loc[17]['Q']+sym+data.loc[17]['A']+"\n2."+test.iloc[i,0]+sym
        resp = generate_res(s).replace(" ", "")
        resp = resp.replace("\n", "")
        one_shot_1.loc[len(one_shot_1)] = [test.iloc[i,0], test.iloc[i,1], resp]
        time.sleep(1.5)

In [ ]:
# one-shot learning with example 14
one_shot_2 = pd.DataFrame(columns=["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
        s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the example:\n1."\
            +data.loc[14]['Q']+sym+data.loc[14]['A']+"\n2."+test.iloc[i,0]+sym
        resp = generate_res(s).replace(" ", "")
        resp = resp.replace("\n", "")
        one_shot_2.loc[len(one_shot_2)] = [test.iloc[i,0], test.iloc[i,1], resp]
        time.sleep(1.5)

In [ ]:
# two-shot using examples 12 and 14
two_shot_1 = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[12]['Q']+sym+data.loc[12]['A']+"\n2."\
            +data.loc[14]['Q']+sym+data.loc[14]['A']+"\n3."+test.iloc[i,0]+sym
    resp = generate_res(s).replace(" ", "")
    resp = resp.replace("\n", "")
    two_shot_1.loc[len(two_shot_1)] = [test.iloc[i,0], test.iloc[i,1], resp]
    time.sleep(1.5)

In [ ]:
# two-shot using examples 17 and 5
two_shot_2 = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[17]['Q']+sym+data.loc[17]['A']+"\n2."\
            +data.loc[5]['Q']+sym+data.loc[5]['A']+"\n3."+test.iloc[i,0]+sym
    resp = generate_res(s).replace(" ", "")
    resp = resp.replace("\n", "")
    two_shot_2.loc[len(two_shot_2)] = [test.iloc[i,0], test.iloc[i,1], resp]
    time.sleep(1.5)

In [ ]:
# two-shot using examples 17 and 12
two_shot_3 = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[17]['Q']+sym+data.loc[17]['A']+"\n2."\
            +data.loc[12]['Q']+sym+data.loc[12]['A']+"\n3."+test.iloc[i,0]+sym
    resp = generate_res(s).replace(" ", "")
    resp = resp.replace("\n", "")
    two_shot_3.loc[len(two_shot_3)] = [test.iloc[i,0], test.iloc[i,1], resp]
    time.sleep(1.5)

In [ ]:
# three-shot using examples (25,12,14)
few_shot_1 = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[25]['Q']+sym+data.loc[25]['A']+"\n2."\
            +data.loc[12]['Q']+sym+data.loc[12]['A']+"\n3."\
            +data.loc[14]['Q']+sym+data.loc[14]['A']+"\n4."+test.iloc[i,0]+sym
    resp = generate_res(s)
    resp = resp.replace("\n","")
    few_shot_1.loc[len(few_shot_1)] = [test.iloc[i,0], test.iloc[i,1], resp]
    time.sleep(1.5)

In [ ]:
# three-shot using examples (17,5,14)
few_shot_2 = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[17]['Q']+sym+data.loc[17]['A']+"\n2."\
            +data.loc[5]['Q']+sym+data.loc[5]['A']+"\n3."\
            +data.loc[14]['Q']+sym+data.loc[14]['A']+"\n4."+test.iloc[i,0]+sym
    resp = generate_res(s)
    resp = resp.replace("\n","")
    few_shot_2.loc[len(few_shot_2)] = [test.iloc[i,0], test.iloc[i,1], resp]
    time.sleep(1.5)

In [ ]:
# three-shot using examples (17,12,14)
few_shot_3 = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[17]['Q']+sym+data.loc[17]['A']+"\n2."\
            +data.loc[12]['Q']+sym+data.loc[12]['A']+"\n3."\
            +data.loc[14]['Q']+sym+data.loc[14]['A']+"\n4."+test.iloc[i,0]+sym
    resp = generate_res(s)
    resp = resp.replace("\n","")
    few_shot_3.loc[len(few_shot_3)] = [test.iloc[i,0], test.iloc[i,1], resp]
    time.sleep(1.5)

In [ ]:
# three-shot using examples (25,5,14)
few_shot_4 = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[25]['Q']+sym+data.loc[25]['A']+"\n2."\
            +data.loc[5]['Q']+sym+data.loc[5]['A']+"\n3."\
            +data.loc[14]['Q']+sym+data.loc[14]['A']+"\n4."+test.iloc[i,0]+sym
    resp = generate_res(s)
    resp = resp.replace("\n","")
    few_shot_4.loc[len(few_shot_4)] = [test.iloc[i,0], test.iloc[i,1], resp]
    time.sleep(1.5)

In [ ]:
# accuracy calculation

#temp_test.loc[len(temp_test)] = ["17", acc_calculation(one_shot_1.iloc[:,1:])]
#temp_test.loc[len(temp_test)] = ["14", acc_calculation(one_shot_2.iloc[:,1:])]
#temp_test.loc[len(temp_test)] = ["(12, 14)", acc_calculation(two_shot_1.iloc[:,1:])]
#temp_test.loc[len(temp_test)] = ["(17, 5)", acc_calculation(two_shot_2.iloc[:,1:])]
#temp_test.loc[len(temp_test)] = ["(17, 12)", acc_calculation(two_shot_3.iloc[:,1:])]
#temp_test.loc[len(temp_test)] = ["(25, 12, 14)", acc_calculation(few_shot_1.iloc[:,1:])]
#temp_test.loc[len(temp_test)] = ["(17, 5, 14)", acc_calculation(few_shot_2.iloc[:,1:])]
#temp_test.loc[len(temp_test)] = ["(17, 12, 14)", acc_calculation(few_shot_3.iloc[:,1:])]
#temp_test.loc[len(temp_test)] = ["(25, 5, 14)", acc_calculation(few_shot_4.iloc[:,1:])]

In [ ]:
# four-shot
few_shot_5 = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(test)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[17]['Q']+sym+data.loc[17]['A']+"\n2."\
            +data.loc[5]['Q']+sym+data.loc[5]['A']+"\n3."\
            +data.loc[12]['Q']+sym+data.loc[12]['A']+"\n4."\
            +data.loc[14]['Q']+sym+data.loc[14]['A']+"\n5."+test.iloc[i,0]+sym
    resp = generate_res(s)
    resp = resp.replace("\n","")
    few_shot_5.loc[len(few_shot_5)] = [test.iloc[i,0], test.iloc[i,1], resp]
    time.sleep(1.5)

# More generations

### Four and five shot learning

In [ ]:
# four-shot learning
combs = list(itertools.combinations(list(range(0,5)),4))
for x,y,z,k in combs:
    for i in range(len(cv)):
        s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +train.iloc[x,0]+sym+train.iloc[x,1]+"\n2."\
            +train.iloc[y,0]+sym+train.iloc[y,1]+"\n3."\
            +train.iloc[z,0]+sym+train.iloc[z,1]+"\n4."\
            +train.iloc[k,0]+sym+train.iloc[k,1]+"\n5."+cv.iloc[i,0]+sym
        resp = generate_res(s)
        resp = resp.replace("\n","")
        results.loc[len(results)] = [(train.iloc[x].name, train.iloc[y].name, train.iloc[z].name, train.iloc[k].name), cv.iloc[i,0], cv.iloc[i,1], resp]
        time.sleep(1.5)

In [ ]:
# five-shot
results = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(rest)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +data.loc[25]['Q']+sym+data.loc[25]['A']+"\n2."\
            +data.loc[17]['Q']+sym+data.loc[17]['A']+"\n3."\
            +data.loc[5]['Q']+sym+data.loc[5]['A']+"\n4."\
            +data.loc[12]['Q']+sym+data.loc[12]['A']+"\n5."\
            +data.loc[14]['Q']+sym+data.loc[14]['A']+"\n6."+rest.iloc[i,0]+sym
    resp = generate_res(s)
    resp = resp.replace("\n","")
    results.loc[len(results)] = [rest.iloc[i,0], rest.iloc[i,1], resp]
    time.sleep(1.5)

### three-shot learning with examples from S&D, POR, MO categories

In [ ]:
q1 = "Tom and Jerry drove 271 total miles in 6 hours. Tom always drives 38 mph and Jerry always drives 59 mph. It's a wonder they can manage to remain friends. How many hours did Tom drive and how many hours did Jerry drive?"
a1 = "x+y=6;38*x+59*y=271"
q2 = "A class of 35 consists of students who either have red or yellow shirts. If there are 15 more people with red shirts than there are people with yellow shirts, how many of the students have red shirts?"
a2 = "x+y=35;x=y+15"
q3 = "The length of a rectangle is 7 units more than the width, and the perimeter is 35 units more than the length. What are the rectangle's dimensions?"
a3 = "2*x+2*y-35=x;x-7=y"
q4 = "The price, x, of an entertainment system at Extreme Electronics is $220 less than twice the price, y, of the same system at Ultra Electronics. The difference in price between the system at Extreme Electronics and Ultra Electronics is $175. Which system of linear equations can be used to determine the price of the system at each store?"
a4 = "x+220=2*y;x-y=175"
q5 = "Shana invests some money in an account that earns 5% simple interest and three times that amount in an account that earns 7% simple interest. If the total interest is $390 at the end of one year, find the amount invested in each account. "
a5 = "3*x=y;0.05*x+0.07*y=390"

In [ ]:
results = pd.DataFrame(columns = ["Question","Right Answer","Generated Answer"])
for i in range(len(d)):
    s = "Extract a system of two linear equations in terms of x and y from the question. Separate the equations with semicolon like given in the examples:\n1."\
            +q1+sym+a1+"\n2."\
            +q2+sym+a2+"\n3."\
            +q3+sym+a3+"\n4."\
            +q4+sym+a4+"\n5."\
            +q5+sym+a5+"\n6."+d.iloc[i,0]+sym
    resp = generate_res(s)
    resp = resp.replace("\n","")
    results.loc[len(results)] = [d.iloc[i,0], d.iloc[i,1], resp]
    time.sleep(1.5)

# Traning set (for fine-tuning) preparation

In [ ]:
df = pd.read_excel("FTSet.xlsx", sheet_name="T20", header=None, usecols="A,B", names=["Q","A"])

In [ ]:
def is_float(ele):
    try:
        float(ele)
        return True
    except ValueError:
        return False

In [ ]:
# generate 4 copies for each pair of Q&A
for index, row in df.iterrows():
    # dataframe to store original and generated question and answer pairs
    odf.loc[len(odf)]=[row["Q"],row["A"]]
    
    # numbers (including numerical words) need to be manually replaced by placeholders
    # values need to be input separately
    print(row["Q"])
    print(row["A"])
    q = input("Q template:\n")
    a = input("A template:\n")
    vals = input("Values:\n")
    ls = vals.split("//")
    combs = [tuple(ls)]
    
    i=0
    while i<4:
        t = list()
        tq = q
        ta = a
        for j in range(len(ls)):
            # handles integers
            if ls[j].isdigit():
                temp = random.randint(int(ls[j])-2, int(ls[j])+3)
                temp = str(temp)
                ta = ta.replace("["+str(j)+"]",temp)
            # handles floats
            elif is_float(ls[j]):
                dig = ls[j].split(".")[1]
                temp = round(random.uniform(float(ls[j])-0.7, float(ls[j])+0.5),len(dig))
                temp = str(temp)
                ta = ta.replace("["+str(j)+"]",temp)
            # handles percentages
            elif "%" in ls[j]:
                num = ls[j][:-1]
                if num.isdigit():
                    newnum = random.randint(int(num)-2, int(num)+3)
                elif is_float(num):
                    dig = num.split(".")[1]
                    newnum = round(random.uniform(float(num)-0.7, float(num)+0.5),len(dig))
                if newnum<0:
                    newnum = (-1)*newnum
                ta = ta.replace("["+str(j)+"]",str(newnum/100))
                temp = str(newnum)+"%"
            # handles numerical texts
            else:
                num = w2n.word_to_num(ls[j])
                if isinstance(num, int):
                    newnum = random.randint(num-2, num+3)
                elif isinstance(num, float):
                    dig = ls[j].split(".")[1]
                    newnum = round(random.uniform(num-0.7, num+0.5),len(dig))
                ta = ta.replace("["+str(j)+"]",str(newnum))
                temp = num2words(newnum)
                if ls[j][0].isupper():
                    temp = temp.capitalize()
            t.append(temp)
            tq = tq.replace("["+str(j)+"]",temp)
        # check for duplicates
        if tuple(t) not in combs:
            odf.loc[len(odf)]=[tq,ta]
            combs.append(tuple(t))
            i += 1

In [ ]:
odf.to_excel()